In [1]:
import datetime
import decimal
import pydantic
import enum
from typing import *
import warnings
import QuantLib as ql
import pandas as pd
import numpy as np
import logging

import finsec as fs

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    # filename='my_application.log',
    # filemode='a'
)

In [2]:
# fs.fixed_income_objs.Period(period='1y').model_dump()

## autoeload info:
%load_ext autoreload
%autoreload 2

In [3]:
# BusinessDayConvention.following.to_ql()
# BusinessDayConvention.from_ql(ql.Following)

In [4]:
import json
import functools
from operator import add, sub, mul

T = TypeVar('T')
ListOrT = Union[List[T], T]

ql.Settings.instance().evaluationDate = ql.Date(3, 1, 2025)

usd_ccy = fs.FiatCurrency( ticker='USD', gsid='USD')
usd = usd_ccy
# usd = fs.create_reference_from_security(usd_ccy)

eur_ccy = fs.FiatCurrency( ticker='EUR', gsid='EUR')
eur = eur_ccy
# eur = fs.create_reference_from_security(eur_ccy)

In [5]:
acc1 = fs.AccrualInfo(
    start=datetime.date(2025,1,1),
    # end=datetime.date(2025,11,5),
    end='3m',
    period='1m',
    dc=fs.DayCount.Thirty360,
    front_stub_not_back=False,
)
# acc1.as_ql().dates()
print(len(acc1))
acc1.schedule().to_df()

3


,start,end,frac
0,2025-01-01,2025-02-01,0.083333
1,2025-02-01,2025-03-01,0.083333
2,2025-03-01,2025-04-01,0.083333


In [6]:
print(json.dumps(json.loads(acc1.model_dump_json()), indent=4))
fs.AccrualInfo.model_validate_json(acc1.model_dump_json())

{
    "start": "2025-01-01",
    "end": {
        "period": "3M"
    },
    "dc": "30/360",
    "freq": null,
    "cal_accrual": "null",
    "cal_pay": "null",
    "period": {
        "period": "1M"
    },
    "bdc": "F",
    "front_stub_not_back": false,
    "eom": false
}


AccrualInfo(start=datetime.date(2025, 1, 1), end=Period("3M"), dc=<DayCount.Thirty360: '30/360'>, freq=None, cal_accrual=<Calendar.NULL: 'null'>, cal_pay=<Calendar.NULL: 'null'>, period=Period("1M"), bdc=<BusinessDayConvention.following: 'F'>, front_stub_not_back=False, eom=False)

In [7]:
expr1 = fs.FixedRate(rate=decimal.Decimal('0.05')) * (decimal.Decimal('1')/10) + 10

expr1.get_fixing(None), expr1.is_constant

(Decimal('10.005'), True)

In [8]:
dc = ql.Actual360()

# for x in dir(dc):
#     print(x)

In [9]:
expr1.model_dump()

{'components': [{}, {}], 'operator': <ExprOperator.ADD: 'ADD'>}

In [10]:
ntnl = 1_000_000
fixleg1 = fs.Leg(
    ccy=usd,
    notional=ntnl,
    cpn=fs.FixedRate(rate=decimal.Decimal('0.01')),
    acc=fs.AccrualInfo(
        start=datetime.date(2025,1,1),
        end=datetime.date(2026,1,1),
        dc=fs.DayCount.Thirty360,
        freq=12,
        bdc=fs.BusinessDayConvention.unadjusted,
    )
)
# print(fixleg1)

bnd1 = fs.Bond(
    notional=ntnl,
    leg=fixleg1,
    # settle: datetime.date
    settle_days=1
)
print(bnd1)
ql_bnd1 = bnd1.as_quantlib()

notional=Decimal('1000000') leg=Leg(ccy=Security(gsid=[USD], ticker=[USD], security_type=[CURRENCY], security_subtype=[NATIONAL_FIAT]), notional=Decimal('1000000'), cpn=FixedRate(rate=Decimal('0.01'), is_constant=True, is_float=False), acc=AccrualInfo(start=datetime.date(2025, 1, 1), end=datetime.date(2026, 1, 1), dc=<DayCount.Thirty360: '30/360'>, freq=Decimal('12'), cal_accrual=<Calendar.NULL: 'null'>, cal_pay=<Calendar.NULL: 'null'>, period=None, bdc=<BusinessDayConvention.unadjusted: 'U'>, front_stub_not_back=True, eom=False), pay_delay=None, log=<Logger Leg (DEBUG)>) settle_days=1 credit_index=None face=100 settle=None redemption=None log=<Logger Bond (DEBUG)>


In [11]:
bnd1.cashflows_df().style.format({'amount':'{0:,.2f}'})

2025-09-26 16:46:43,462 - DEBUG - matplotlib - matplotlib data path: /config/workspace/venv/lib/python3.12/site-packages/matplotlib/mpl-data
2025-09-26 16:46:43,468 - DEBUG - matplotlib - CONFIGDIR=/config/.config/matplotlib
2025-09-26 16:46:43,484 - DEBUG - matplotlib - interactive is False
2025-09-26 16:46:43,485 - DEBUG - matplotlib - platform is linux
2025-09-26 16:46:43,606 - DEBUG - matplotlib - CACHEDIR=/config/.cache/matplotlib
2025-09-26 16:46:43,608 - DEBUG - matplotlib.font_manager - Using fontManager instance from /config/.cache/matplotlib/fontlist-v390.json


,start,end,frac,date,amount,has_occurred,rate
0,2025-01-01,2025-02-01,0.083333,2025-02-01,833.33,False,0.010000
1,2025-02-01,2025-03-01,0.083333,2025-03-01,833.33,False,0.010000
2,2025-03-01,2025-04-01,0.083333,2025-04-01,833.33,False,0.010000
3,2025-04-01,2025-05-01,0.083333,2025-05-01,833.33,False,0.010000
4,2025-05-01,2025-06-01,0.083333,2025-06-01,833.33,False,0.010000
5,2025-06-01,2025-07-01,0.083333,2025-07-01,833.33,False,0.010000
6,2025-07-01,2025-08-01,0.083333,2025-08-01,833.33,False,0.010000
7,2025-08-01,2025-09-01,0.083333,2025-09-01,833.33,False,0.010000
8,2025-09-01,2025-10-01,0.083333,2025-10-01,833.33,False,0.010000
9,2025-10-01,2025-11-01,0.083333,2025-11-01,833.33,False,0.010000


In [12]:
bnd1.model_dump()

/config/workspace/venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `decimal` - serialized value may not be as expected [input_value=100, input_type=int])
  return self.__pydantic_serializer__.to_python(


{'notional': Decimal('1000000'),
 'leg': {'ccy': {'gsid': 'USD',
   'ticker': 'USD',
   'security_type': <SecurityType.CURRENCY: 2>,
   'security_subtype': <SecuritySubtype.NATIONAL_FIAT: 505>,
   'identifiers': [],
   'primary_exchange': None,
   'denominated_ccy': None,
   'issuer': None,
   'description': None,
   'website': None,
   'as_of_date': None,
   'version_id': None},
  'notional': Decimal('1000000'),
  'cpn': {'rate': Decimal('0.01'), 'is_constant': True, 'is_float': False},
  'acc': {'start': datetime.date(2025, 1, 1),
   'end': datetime.date(2026, 1, 1),
   'dc': <DayCount.Thirty360: '30/360'>,
   'freq': Decimal('12'),
   'cal_accrual': <Calendar.NULL: 'null'>,
   'cal_pay': <Calendar.NULL: 'null'>,
   'period': None,
   'bdc': <BusinessDayConvention.unadjusted: 'U'>,
   'front_stub_not_back': True,
   'eom': False},
  'pay_delay': None},
 'settle_days': 1,
 'credit_index': None,
 'face': 100,
 'settle': None,
 'redemption': None}

In [13]:
bnd2 = fs.Bond.model_validate_json(bnd1.model_dump_json())
bnd2.model_dump()

/config/workspace/venv/lib/python3.12/site-packages/pydantic/main.py:519: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `decimal` - serialized value may not be as expected [input_value=100, input_type=int])
  return self.__pydantic_serializer__.to_json(


{'notional': Decimal('1000000'),
 'leg': {'ccy': {'gsid': 'USD',
   'ticker': 'USD',
   'security_type': <SecurityType.CURRENCY: 2>,
   'security_subtype': <SecuritySubtype.NATIONAL_FIAT: 505>,
   'identifiers': [],
   'primary_exchange': None,
   'denominated_ccy': None,
   'issuer': None,
   'description': None,
   'website': None,
   'as_of_date': None,
   'version_id': None},
  'notional': Decimal('1000000'),
  'cpn': {'rate': Decimal('0.01'), 'is_constant': True, 'is_float': False},
  'acc': {'start': datetime.date(2025, 1, 1),
   'end': datetime.date(2026, 1, 1),
   'dc': <DayCount.Thirty360: '30/360'>,
   'freq': Decimal('12'),
   'cal_accrual': <Calendar.NULL: 'null'>,
   'cal_pay': <Calendar.NULL: 'null'>,
   'period': None,
   'bdc': <BusinessDayConvention.unadjusted: 'U'>,
   'front_stub_not_back': True,
   'eom': False},
  'pay_delay': None},
 'settle_days': 1,
 'credit_index': None,
 'face': Decimal('100'),
 'settle': None,
 'redemption': None}

In [14]:
bnd1.model_dump()

{'notional': Decimal('1000000'),
 'leg': {'ccy': {'gsid': 'USD',
   'ticker': 'USD',
   'security_type': <SecurityType.CURRENCY: 2>,
   'security_subtype': <SecuritySubtype.NATIONAL_FIAT: 505>,
   'identifiers': [],
   'primary_exchange': None,
   'denominated_ccy': None,
   'issuer': None,
   'description': None,
   'website': None,
   'as_of_date': None,
   'version_id': None},
  'notional': Decimal('1000000'),
  'cpn': {'rate': Decimal('0.01'), 'is_constant': True, 'is_float': False},
  'acc': {'start': datetime.date(2025, 1, 1),
   'end': datetime.date(2026, 1, 1),
   'dc': <DayCount.Thirty360: '30/360'>,
   'freq': Decimal('12'),
   'cal_accrual': <Calendar.NULL: 'null'>,
   'cal_pay': <Calendar.NULL: 'null'>,
   'period': None,
   'bdc': <BusinessDayConvention.unadjusted: 'U'>,
   'front_stub_not_back': True,
   'eom': False},
  'pay_delay': None},
 'settle_days': 1,
 'credit_index': None,
 'face': 100,
 'settle': None,
 'redemption': None}

In [15]:
from functools import reduce
from operator import add

# help(reduce)
fs.Calendar.US_SOFR.bump(datetime.date(2025, 1, 1), '10y')

[datetime.date(2035, 1, 2)]

In [16]:
swp_ntnl = 1_000_000
swp_start = datetime.date(2025,1,3)
swp_end = '3y'
swp_index = 'SOFR'
swp = fs.Swap.make_ois(
    ccy=usd,
    start=swp_start,
    end=swp_end,
    rate=5.0 / 100,
    dc_fix=fs.DayCount.Actual360,
    dc_float=fs.DayCount.Actual360,
    freq_fix=1,
    freq_float=1,
    index=swp_index,
    cal_pay=fs.Calendar.US_SOFR,
    notional=swp_ntnl,
    pay_delay=fs.Period(period='2d'),
)
display(swp.model_dump())

{'legs': ({'ccy': {'gsid': 'USD',
    'ticker': 'USD',
    'security_type': <SecurityType.CURRENCY: 2>,
    'security_subtype': <SecuritySubtype.NATIONAL_FIAT: 505>,
    'identifiers': [],
    'primary_exchange': None,
    'denominated_ccy': None,
    'issuer': None,
    'description': None,
    'website': None,
    'as_of_date': None,
    'version_id': None},
   'notional': Decimal('1000000'),
   'cpn': {'rate': Decimal('0.05'), 'is_constant': True, 'is_float': False},
   'acc': {'start': datetime.date(2025, 1, 3),
    'end': {'period': '3Y'},
    'dc': <DayCount.Actual360: 'act/360'>,
    'freq': Decimal('1'),
    'cal_accrual': <Calendar.NULL: 'null'>,
    'cal_pay': <Calendar.US_SOFR: 'us/sofr'>,
    'period': None,
    'bdc': <BusinessDayConvention.following: 'F'>,
    'front_stub_not_back': True,
    'eom': False},
   'pay_delay': None},
  {'ccy': {'gsid': 'USD',
    'ticker': 'USD',
    'security_type': <SecurityType.CURRENCY: 2>,
    'security_subtype': <SecuritySubtype.NATIONA

In [17]:
ql_curve_raw = ql.FlatForward(
    0,
    fs.Calendar.US_SOFR.as_ql(),
    ql.QuoteHandle(ql.SimpleQuote(
        0.05,
    )),
    ql.Actual360(),
)

ql_curve = ql.YieldTermStructureHandle(ql_curve_raw)

swp_engine = ql.DiscountingSwapEngine(ql_curve)
sofr_idx = ql.Sofr(ql_curve)
swp_lookup = {
    'SOFR': sofr_idx,
}


In [18]:
ql_swp = swp.as_quantlib( index_lookup=swp_lookup )

ql_swp.setPricingEngine(swp_engine)
ql_swp.fairRate()

0.05114746391826529

In [19]:
swp.fixed_leg.cashflows_df(index_lookup=swp_lookup)

,start,end,frac,amount,has_occurred,rate
date,,,,,,
2026-01-05,2025-01-03,2026-01-03,1.013889,50694.444444,False,0.05
2027-01-04,2026-01-03,2027-01-03,1.013889,50694.444444,False,0.05
2028-01-03,2027-01-03,2028-01-03,1.013889,50694.444444,False,0.05


In [20]:
swp.cashflows_df( index_lookup=swp_lookup )

,start,end,frac,amount,has_occurred,rate,fix
date,,,,,,,
2026-01-05,2025-01-03,2026-01-03,1.013889,50694.444444,False,0.050000,True
2026-01-05,2025-01-03,2026-01-03,1.013889,-52001.419591,False,0.051289,False
2027-01-04,2026-01-03,2027-01-03,1.013889,50694.444444,False,0.050000,True
2027-01-04,2026-01-03,2027-01-03,1.013889,-51709.247697,False,0.051001,False
2028-01-03,2027-01-03,2028-01-03,1.013889,50694.444444,False,0.050000,True
2028-01-03,2027-01-03,2028-01-03,1.013889,-51855.298137,False,0.051145,False


In [21]:
# help(ql.MakeOIS)
ql_swp2 = ql.MakeOIS(
    ql.Period(swp_end),
    sofr_idx,
    0.05,
    paymentLag=2,
    paymentCalendar=ql.UnitedStates( ql.UnitedStates.SOFR ),
    paymentAdjustmentConvention=ql.Following,
    endOfMonth=False,
)
ql_swp2.setPricingEngine(swp_engine)
ql_swp2.fairRate()

0.05128905136099185

In [22]:
print(f'''
my swap: {ql_swp.fairRate():,.3%}
makeois: {ql_swp2.fairRate():,.3%}
diff: {(ql_swp.fairRate()-ql_swp2.fairRate())*10000:+,.2f}bp
''')



my swap: 5.115%
makeois: 5.129%
diff: -1.42bp



In [23]:
# help(ql.DiscountingSwapEngine)
# help(ql.DatedOISRateHelper)
# swp.fixed_leg.as_quantlib()

display(swp.fixed_leg.acc.model_dump())
display(swp.float_leg.acc.model_dump())
# swp.fixed_leg.acc.schedule().model_dump()

fs.Quote(quote=0).handle

{'start': datetime.date(2025, 1, 3),
 'end': {'period': '3Y'},
 'dc': <DayCount.Actual360: 'act/360'>,
 'freq': Decimal('1'),
 'cal_accrual': <Calendar.NULL: 'null'>,
 'cal_pay': <Calendar.US_SOFR: 'us/sofr'>,
 'period': None,
 'bdc': <BusinessDayConvention.following: 'F'>,
 'front_stub_not_back': True,
 'eom': False}

{'start': datetime.date(2025, 1, 3),
 'end': {'period': '3Y'},
 'dc': <DayCount.Actual360: 'act/360'>,
 'freq': Decimal('1'),
 'cal_accrual': <Calendar.NULL: 'null'>,
 'cal_pay': <Calendar.US_SOFR: 'us/sofr'>,
 'period': None,
 'bdc': <BusinessDayConvention.following: 'F'>,
 'front_stub_not_back': True,
 'eom': False}

<QuantLib.QuantLib.QuoteHandle; proxy of <Swig Object of type 'Handle< Quote > *' at 0x7f38ae1b9fe0> >

In [24]:
swp.as_quantlib_helper(index_lookup=swp_lookup)


2025-09-26 16:46:44,244 - DEBUG - Swap - Building actual ql helper


(Quote(quote=<QuantLib.QuantLib.SimpleQuote; proxy of <Swig Object of type 'ext::shared_ptr< SimpleQuote > *' at 0x7f38ae1bb930> >),
 <QuantLib.QuantLib.DatedOISRateHelper; proxy of <Swig Object of type 'ext::shared_ptr< DatedOISRateHelper > *' at 0x7f38ae1bbb10> >)

In [25]:
help(ql.IborIndex)

Help on class IborIndex in module QuantLib.QuantLib:

class IborIndex(InterestRateIndex)
 |  IborIndex(*args)
 |
 |  Proxy of C++ IborIndex class.
 |
 |  Method resolution order:
 |      IborIndex
 |      InterestRateIndex
 |      Index
 |      Observable
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __init__(self, *args)
 |      __init__(IborIndex self, std::string const & familyName, Period tenor, Integer settlementDays, Currency currency, Calendar calendar, BusinessDayConvention convention, bool endOfMonth, DayCounter dayCounter, YieldTermStructureHandle h=Handle< YieldTermStructure >()) -> IborIndex
 |
 |  __repr__ = _swig_repr(self) from QuantLib.QuantLib
 |
 |  businessDayConvention(self)
 |      businessDayConvention(IborIndex self) -> BusinessDayConvention
 |
 |  clone(self, arg2)
 |      clone(IborIndex self, YieldTermStructureHandle arg2) -> ext::shared_ptr< IborIndex >
 |
 |  endOfMonth(self)
 |      endOfMonth(IborIndex self) -> bool
 |
 |  forwardingTermStruc

## bootstrap a treasury curve

In [26]:
class SingleCurveConfig(pydantic.BaseModel):
    name: Hashable
    date: datetime.date
    settle_days: int
    dc: fs.DayCount
    cal: fs.Calendar

    # parallel_bump:fs.Quote = pydantic.Field(default_factory=fs.Quote)
    resets: Dict[datetime.date, decimal.Decimal] = pydantic.Field(default_factory=dict)
    helpers: Dict[str, Tuple[fs.Quote, Any]] = pydantic.Field(default_factory=dict)

    ql_curve_raw: ql.YieldTermStructure|None=None
    handle: ql.RelinkableYieldTermStructureHandle=pydantic.Field(default_factory=ql.RelinkableYieldTermStructureHandle)
    index: ql.InterestRateIndex|None=None

    class Config:
        arbitrary_types_allowed = True
    
    def benchmarks(self)->Dict[str, fs.Quote]:
        return { n:q[0] for n,q in self.helpers.items() }

    def add_helper(self, name:str, quote:fs.Quote, helper:Any):
        self.helpers[name] = (quote, helper)
    
    def add_reset(self, dt:datetime.date, rate:decimal.Decimal):
        self.resets[dt] = rate
        if self.index is not None:
            self.index.addFixing( ql.Date.from_date(dt), float(rate))
    
    def build_quantlib_index(self):
        usd = ql.USDCurrency()
        self.index = ql.OvernightIndex(
            self.name,
            self.settle_days,
            usd,
            self.cal.as_ql(),
            self.dc.as_ql,
            self.handle
        )
        self.index.addFixings(
            [ql.Date.from_date(x) for x in self.resets.keys()],
            [float(x) for x in self.resets.values()],
        )

    def build_quantlib_curve(self):
        instruments = [x[1] for x in self.helpers.values()]
        self.ql_curve_raw = ql.PiecewiseFlatForward(
            ql.Date.from_date(self.date),
            instruments,
            self.dc.as_ql,
        )
        self.handle.linkTo(self.ql_curve_raw)
    
    # def plot_forwards()

# class CurveInterp(pydantic.BaseModel):
#     order: List[Literal[0,1,2]]
#     joint_date:

In [27]:
tsy_curve = SingleCurveConfig(
    name='USGOVT',
    date=datetime.date(2025, 1, 1),
    settle_days=1,
    dc=fs.DayCount.ActualActual,
    cal=fs.Calendar.US_GovernmentBond,
)

In [28]:
bond_tenors = ['1y','2y','5y','10y']
bonds = [
    fs.Bond(
        credit_index='USGOVT',
        notional=ntnl,
        leg=fs.Leg(
            ccy=usd,
            notional=100,
            cpn=fs.FixedRate(rate=5.0 / 100),
            acc=fs.AccrualInfo(
                start=datetime.date(2025,1,1),
                end=ten,
                dc=fs.DayCount.ActualActual,
                freq=2,
                bdc=fs.BusinessDayConvention.modified_following,
            )
        ),
        settle_days=1
    )
    for ten in bond_tenors
]

# bond_prices = [100] * len(bond_tenors)
bond_prices = [
    100,
    99,
    98,
    97,
]
bond_helpers = [bnd.as_quantlib_helper() for bnd in bonds]
for px, q in zip(bond_prices, bond_helpers):
    q[0].setValue(px)

In [29]:
for ten, qh in zip(bond_tenors, bond_helpers):
    q,h = qh
    tsy_curve.add_helper( ten, q, h)

In [30]:
tsy_curve.build_quantlib_curve()
tsy_curve.ql_curve_raw.enableExtrapolation()

In [31]:
# tsy_curve.plot(datetime.date(2025,1,1), datetime.date(2035,1,1))

In [32]:
# swp_crv0_ql = ql.PiecewiseFlatForward(
# )

# help(ql.PiecewiseFlatForward)
# PiecewiseFlatForward self
#     Date referenceDate
#     RateHelperVector instruments
#     DayCounter dayCounter
#     QuoteHandleVector jumps=std::vector< Handle< Quote > >()
#     DateVector jumpDates=std::vector< Date >()
#     BackwardFlat i=BackwardFlat()
#     IterativeBootstrap b=_IterativeBootstrap()

for x in dir(ql):
    if 'piecewise' in x.lower():
        print(x)

PiecewiseConstantCorrelation
PiecewiseConstantParameter
PiecewiseConvexMonotoneForward
PiecewiseConvexMonotoneZero
PiecewiseCubicZero
PiecewiseFlatForward
PiecewiseFlatHazardRate
PiecewiseForwardSpreadedTermStructure
PiecewiseKrugerLogDiscount
PiecewiseKrugerZero
PiecewiseLinearForward
PiecewiseLinearForwardSpreadedTermStructure
PiecewiseLinearZero
PiecewiseLogCubicDiscount
PiecewiseLogLinearDiscount
PiecewiseLogMixedLinearCubicDiscount
PiecewiseLogParabolicCubicDiscount
PiecewiseMonotonicLogParabolicCubicDiscount
PiecewiseMonotonicParabolicCubicZero
PiecewiseNaturalCubicZero
PiecewiseNaturalLogCubicDiscount
PiecewiseParabolicCubicZero
PiecewiseSplineCubicDiscount
PiecewiseTimeDependentHestonModel
PiecewiseYoYInflation
PiecewiseZeroInflation
PiecewiseZeroSpreadedTermStructure


In [33]:
import salt.curve as crv

today = datetime.date(2025,1,3)
ql.Settings.instance().evaluationDate = ql.Date.from_date(today)

spot_dt = fs.Calendar.US_SOFR.bump( today, '2d')[0]
print(spot_dt)

index_name = 'SOFR'

sofr_curve = crv.SingleCurveConfig(
    ccy=usd,
    name=index_name,

    date=today,
    # date=datetime.date(2025, 1, 7),
    settle_days=2,
    # date=spot_dt,
    # settle_days=0,

    dc=fs.DayCount.Actual360,
    cal=fs.Calendar.US_SOFR,
)
estr_curve = crv.SingleCurveConfig(
    ccy=eur,
    name='ESTR',
    date=today,
    settle_days=2,
    dc=fs.DayCount.Actual360,
    cal=fs.Calendar.EU_TARGET,
)
# print(sofr_curve)
curve = crv.MultiCurve(
    curves={
        index_name:sofr_curve,
        'ESTR':estr_curve,
    },
    default_curve_by_ccy={
        usd.gsid:'SOFR',
        eur.gsid:'ESTR',
    }
)
print(curve)
print(curve.index_lookup)

# bm_rates = 
# swp_ntnl = 1_000_000

swps = []
quotes = []
rate_mult = 100
bms = dict()
for ten, r in [
        ('1y',  3.610),
        ('2y',  3.331),
        ('3y',  3.270),
        ('5y',  3.311),
        ('7y',  3.427),
        ('10y', 3.608),
        ('12y', 3.717),
        ('20y', 3.943),
        ('30y', 3.922),
        ('40y', 3.817),
    ]:
    swp = fs.Swap.make_ois(
        ccy=usd,
        start=spot_dt,
        end=fs.Period(period=ten),
        rate=r/100,
        dc_fix=fs.DayCount.Actual360,
        dc_float=fs.DayCount.Actual360,
        freq_fix=1,
        freq_float=1,
        index=index_name,
        cal_pay=fs.Calendar.US_SOFR,
        pay_delay=fs.Period(period='2d'),
    )
    quote = fs.Quote(quote=ql.SimpleQuote(r))
    bms[ten] = crv.Benchmark(
        rate=quote,
        instrument=swp,
        risk_mult=1e-2,
        bootstrap_mult=1e-2,
    )

sofr_curve.bms = bms

2025-09-26 16:46:44,490 - DEBUG - SingleCurveConfig - building quantlib index for SOFR
2025-09-26 16:46:44,490 - DEBUG - SingleCurveConfig - Done building quantlib index
2025-09-26 16:46:44,491 - DEBUG - SingleCurveConfig - building quantlib index for ESTR
2025-09-26 16:46:44,492 - DEBUG - SingleCurveConfig - Done building quantlib index
2025-09-26 16:46:44,493 - DEBUG - MultiCurve - Building all curves


2025-01-07
curves={'SOFR': SingleCurveConfig(name='SOFR', ccy=Security(gsid=[USD], ticker=[USD], security_type=[CURRENCY], security_subtype=[NATIONAL_FIAT]), date=datetime.date(2025, 1, 3), settle_days=2, dc=<DayCount.Actual360: 'act/360'>, cal=<Calendar.US_SOFR: 'us/sofr'>, extrapolate=True, virgin=True, bms={}, resets={}, handle=<QuantLib.QuantLib.RelinkableYieldTermStructureHandle; proxy of <Swig Object of type 'RelinkableHandle< YieldTermStructure > *' at 0x7f38ae1b9560> >, ql_curve_raw=None, index=<QuantLib.QuantLib.OvernightIndex; proxy of <Swig Object of type 'ext::shared_ptr< OvernightIndex > *' at 0x7f38adf4f1b0> >, log=<Logger SingleCurveConfig (DEBUG)>), 'ESTR': SingleCurveConfig(name='ESTR', ccy=Security(gsid=[EUR], ticker=[EUR], security_type=[CURRENCY], security_subtype=[NATIONAL_FIAT]), date=datetime.date(2025, 1, 3), settle_days=2, dc=<DayCount.Actual360: 'act/360'>, cal=<Calendar.EU_TARGET: 'eu/target'>, extrapolate=True, virgin=True, bms={}, resets={}, handle=<QuantLi

In [34]:
swps = []
quotes = []
rate_mult = 100
bms = dict()
for ten, r in [
        ('1Y', 2.5),
        ('2Y', 2.6),
        ('5Y', 2.7),
        ('10Y', 2.8),
        ('30Y', 2.9),
    ]:
    swp = fs.Swap.make_ois(
        ccy=eur,
        start=spot_dt,
        end=fs.Period(period=ten),
        rate=r/100,
        dc_fix=fs.DayCount.Actual360,
        dc_float=fs.DayCount.Actual360,
        freq_fix=1,
        freq_float=1,
        index='ESTR',
        cal_pay=fs.Calendar.EU_TARGET,
        # pay_delay=fs.Period(period='2d'),
    )
    quote = fs.Quote(quote=ql.SimpleQuote(r))
    bms[ten] = crv.Benchmark(
        rate=quote,
        instrument=swp,
        risk_mult=1e-2,
        bootstrap_mult=1e-2,
    )

estr_curve.bms = bms
curve.init_all_curves()
# curve = crv.MultiCurve.model_validate_json( curve.model_dump_json() )

2025-09-26 16:46:44,538 - DEBUG - MultiCurve - Building all curves
2025-09-26 16:46:44,539 - DEBUG - SingleCurveConfig - Building quantlib curve
2025-09-26 16:46:44,539 - DEBUG - salt.curve - Benchmark building 
2025-09-26 16:46:44,540 - DEBUG - Swap - Building actual ql helper
2025-09-26 16:46:44,542 - DEBUG - salt.curve - Benchmark building 
2025-09-26 16:46:44,543 - DEBUG - Swap - Building actual ql helper
2025-09-26 16:46:44,546 - DEBUG - salt.curve - Benchmark building 
2025-09-26 16:46:44,547 - DEBUG - Swap - Building actual ql helper
2025-09-26 16:46:44,551 - DEBUG - salt.curve - Benchmark building 
2025-09-26 16:46:44,551 - DEBUG - Swap - Building actual ql helper
2025-09-26 16:46:44,557 - DEBUG - salt.curve - Benchmark building 
2025-09-26 16:46:44,558 - DEBUG - Swap - Building actual ql helper
2025-09-26 16:46:44,565 - DEBUG - salt.curve - Benchmark building 
2025-09-26 16:46:44,566 - DEBUG - Swap - Building actual ql helper
2025-09-26 16:46:44,575 - DEBUG - salt.curve - Benc

In [35]:
# default: 'Any' = PydanticUndefined,
# 	*,
# 	default_factory: 'Callable[[], Any] | Callable[[dict[str, Any]], Any] | None' = PydanticUndefined,
# 	alias: 'str | None' = PydanticUndefined,
# 	alias_priority: 'int | None' = PydanticUndefined,
# 	validation_alias: 'str | AliasPath | AliasChoices | None' = PydanticUndefined,
# 	serialization_alias: 'str | None' = PydanticUndefined,
# 	title: 'str | None' = PydanticUndefined,
# 	field_title_generator: 'Callable[[str, FieldInfo], str] | None' = PydanticUndefined,
# 	description: 'str | None' = PydanticUndefined,
# 	examples: 'list[Any] | None' = PydanticUndefined,
# 	exclude: 'bool | None' = PydanticUndefined,
# 	discriminator: 'str | types.Discriminator | None' = PydanticUndefined,
# 	deprecated: 'Deprecated | str | bool | None' = PydanticUndefined,
# 	json_schema_extra: 'JsonDict | Callable[[JsonDict], None] | None' = PydanticUndefined,
# 	frozen: 'bool | None' = PydanticUndefined,
# 	validate_default: 'bool | None' = PydanticUndefined,
# 	repr: 'bool' = PydanticUndefined,
# 	init: 'bool | None' = PydanticUndefined,
# 	init_var: 'bool | None' = PydanticUndefined,
# 	kw_only: 'bool | None' = PydanticUndefined,
# 	pattern: 'str | typing.Pattern[str] | None' = PydanticUndefined,
# 	strict: 'bool | None' = PydanticUndefined,
# 	coerce_numbers_to_str: 'bool | None' = PydanticUndefined,
# 	gt: 'annotated_types.SupportsGt | None' = PydanticUndefined,
# 	ge: 'annotated_types.SupportsGe | None' = PydanticUndefined,
# 	lt: 'annotated_types.SupportsLt | None' = PydanticUndefined,
# 	le: 'annotated_types.SupportsLe | None' = PydanticUndefined,
# 	multiple_of: 'float | None' = PydanticUndefined,
# 	allow_inf_nan: 'bool | None' = PydanticUndefined,
# 	max_digits: 'int | None' = PydanticUndefined,
# 	decimal_places: 'int | None' = PydanticUndefined,
# 	min_length: 'int | None' = PydanticUndefined,
# 	max_length: 'int | None' = PydanticUndefined,
# 	union_mode: "Literal['smart', 'left_to_right']" = PydanticUndefined,
# 	fail_fast: 'bool | None' = PydanticUndefined,
# 	**extra: 'Unpack[_EmptyKwargs]') -> 'Any'

In [36]:
def show_curve(crv):
    c = crv.currentLink()
    print(c)
    # for x in dir(c):
    #     if x[0] != '_':
    #         print(x)
    dt0 = c.referenceDate().to_date()
    dt1 = c.maxDate().to_date()
    print(f'{dt0.strftime("%d-%b-%y")} -> {dt1.strftime("%d-%b-%y")}')
    # print(c.dates())

# show_curve(sofr_curve.handle)

In [37]:
# hlpr = sofr_curve.bms['1y'].create_helper(curve)
# # dir(hlpr)
# hlpr.earliestDate(), hlpr.latestDate(), hlpr.pillarDate(), hlpr.quote().value()

In [38]:
swp_test_raw = fs.Swap.make_ois(
    ccy=usd,
    start=spot_dt,
    end=fs.Period(period='10y'),
    rate=3.6/100,
    dc_fix=fs.DayCount.Actual360,
    dc_float=fs.DayCount.Actual360,
    freq_fix=1,
    freq_float=1,
    index=index_name,
    cal_pay=fs.Calendar.US_SOFR,
    notional=1_000_000,
    pay_delay=fs.Period(period='2d'),
)
# swp_test = swp_test_raw.as_quantlib(index_lookup=curve.index_lookup)
# swp_test.setPricingEngine(ql.DiscountingSwapEngine(sofr_curve.handle))
# sofr_curve.benchmark_rates()

In [39]:
# dir(multi_risk.ql_obj)
# help(multi_risk.ql_obj.NPV)

In [50]:
logging.getLogger().setLevel(logging.DEBUG)

multi_risk = curve.risk_factory(
    swp_test_raw.risk_builder(
        # funding_index=curve.get_curve_handle('SOFR'),
        # funding_index='SOFR'
    ),
    # eps=1e-5
)
print(f'fair rate: {multi_risk.ql_obj.fairRate():,.3%}')
print('multi risk pv:', multi_risk.pv)
# df = multi_risk.curve['SOFR'].as_df()
# print(df['risk'].sum())
# display(df.style.format({
#     'risk':'{0:,.1f}'
# }))

# multi_risk = curve.risk_factory(
#     swp_test_raw.risk_builder(
#         # funding_index=curve.get_curve_handle('SOFR'),
#         funding_index='SOFR'
#     ),
#     eps=1e-5
# )
print(multi_risk.pv)
df = multi_risk.curve['SOFR'].as_df()
print(df['risk'].sum())
df.style.format({ ('risk','USD'):'{0:,.1f}' })

2025-09-26 16:56:27,916 - DEBUG - MultiCurve - Risk factory
2025-09-26 16:56:27,917 - DEBUG - Swap - funding curve df: 0.223648
2025-09-26 16:56:27,940 - DEBUG - SingleCurveConfig - Building full risk ladder
2025-09-26 16:56:27,941 - DEBUG - SingleCurveConfig - PVing
2025-09-26 16:56:27,941 - DEBUG - SingleCurveConfig - Risking for 1y
2025-09-26 16:56:28,109 - DEBUG - SingleCurveConfig - Risking for 2y


got default funding for ccy USD SOFR


2025-09-26 16:56:28,261 - DEBUG - SingleCurveConfig - Risking for 3y
2025-09-26 16:56:28,424 - DEBUG - SingleCurveConfig - Risking for 5y
2025-09-26 16:56:28,588 - DEBUG - SingleCurveConfig - Risking for 7y
2025-09-26 16:56:28,744 - DEBUG - SingleCurveConfig - Risking for 10y
2025-09-26 16:56:28,883 - DEBUG - SingleCurveConfig - Risking for 12y
2025-09-26 16:56:29,021 - DEBUG - SingleCurveConfig - Risking for 20y
2025-09-26 16:56:29,156 - DEBUG - SingleCurveConfig - Risking for 30y
2025-09-26 16:56:29,289 - DEBUG - SingleCurveConfig - Risking for 40y
2025-09-26 16:56:29,540 - DEBUG - salt.curve - returning dataframe


fair rate: 3.604%
multi risk pv: security=Security(gsid=[USD], ticker=[USD], security_type=[CURRENCY], security_subtype=[NATIONAL_FIAT]) quantity=Decimal('-368.96463503624545')
security=Security(gsid=[USD], ticker=[USD], security_type=[CURRENCY], security_subtype=[NATIONAL_FIAT]) quantity=Decimal('-368.96463503624545')
USD   -842.049794
dtype: float64


,rate,risk
,,USD
1y,3.610000,-0.5
2y,3.331000,0.0
3y,3.270000,0.4
5y,3.311000,-1.8
7y,3.427000,1.4
10y,3.608000,-846.8
12y,3.717000,-0.4
20y,3.943000,5.8
30y,3.922000,0.0


In [51]:
# curve.model_dump()
# 1/0

In [52]:
swp_test_raw = fs.Swap.make_ois(
    ccy=usd,
    start=spot_dt,
    end=fs.Period(period='8y'),
    rate=r/100,
    dc_fix=fs.DayCount.Actual360,
    dc_float=fs.DayCount.Actual360,
    freq_fix=1,
    freq_float=1,
    index=index_name,
    cal_pay=fs.Calendar.US_SOFR,
    notional=20_000_000,
    pay_delay=fs.Period(period='2d'),
)
# swp_test = swp_test_raw.as_quantlib(index_lookup=curve.index_lookup)
# swp_test.setPricingEngine(ql.DiscountingSwapEngine(sofr_curve.handle))
# sofr_curve.benchmark_rates()

multi_risk = curve.risk_factory(
    swp_test_raw.risk_builder(
        # funding_index=curve.get_curve_handle('SOFR'),
        # funding_index='SOFR'
    ),
)
print(multi_risk.pv)
df = multi_risk.curve['SOFR'].as_df()
print(df['risk'].sum())
df.style.format({ ('risk','USD'):'{0:,.1f}' })

2025-09-26 16:56:29,595 - DEBUG - MultiCurve - Risk factory
2025-09-26 16:56:29,596 - DEBUG - Swap - funding curve df: 0.223648
2025-09-26 16:56:29,604 - DEBUG - SingleCurveConfig - Building full risk ladder
2025-09-26 16:56:29,604 - DEBUG - SingleCurveConfig - PVing
2025-09-26 16:56:29,604 - DEBUG - SingleCurveConfig - Risking for 1y


got default funding for ccy USD SOFR


2025-09-26 16:56:29,754 - DEBUG - SingleCurveConfig - Risking for 2y
2025-09-26 16:56:29,907 - DEBUG - SingleCurveConfig - Risking for 3y
2025-09-26 16:56:30,069 - DEBUG - SingleCurveConfig - Risking for 5y
2025-09-26 16:56:30,235 - DEBUG - SingleCurveConfig - Risking for 7y
2025-09-26 16:56:30,390 - DEBUG - SingleCurveConfig - Risking for 10y
2025-09-26 16:56:30,530 - DEBUG - SingleCurveConfig - Risking for 12y
2025-09-26 16:56:30,668 - DEBUG - SingleCurveConfig - Risking for 20y
2025-09-26 16:56:30,804 - DEBUG - SingleCurveConfig - Risking for 30y
2025-09-26 16:56:30,936 - DEBUG - SingleCurveConfig - Risking for 40y
2025-09-26 16:56:31,188 - DEBUG - salt.curve - returning dataframe


security=Security(gsid=[USD], ticker=[USD], security_type=[CURRENCY], security_subtype=[NATIONAL_FIAT]) quantity=Decimal('-840190.6477940413')
USD   -13616.00154
dtype: float64


,rate,risk
,,USD
1y,3.610000,-0.9
2y,3.331000,18.5
3y,3.270000,48.9
5y,3.311000,58.2
7y,3.427000,"-7,909.6"
10y,3.608000,"-5,889.6"
12y,3.717000,18.5
20y,3.943000,40.0
30y,3.922000,-0.0


In [53]:
import salt.fx
# help(ql.DiscountingSwapEngine)

help(ql.FxSwapRateHelper)

Help on class FxSwapRateHelper in module QuantLib.QuantLib:

class FxSwapRateHelper(RateHelper)
 |  FxSwapRateHelper(*args)
 |
 |  Proxy of C++ FxSwapRateHelper class.
 |
 |  Method resolution order:
 |      FxSwapRateHelper
 |      RateHelper
 |      Observable
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __init__(self, *args)
 |      __init__(FxSwapRateHelper self, QuoteHandle fwdPoint, QuoteHandle spotFx, Period tenor, Natural fixingDays, Calendar calendar, BusinessDayConvention convention, bool endOfMonth, bool isFxBaseCurrencyCollateralCurrency, YieldTermStructureHandle collateralCurve, Calendar tradingCalendar=Calendar()) -> FxSwapRateHelper
 |
 |  __repr__ = _swig_repr(self) from QuantLib.QuantLib
 |
 |  adjustmentCalendar(self)
 |      adjustmentCalendar(FxSwapRateHelper self) -> Calendar
 |
 |  businessDayConvention(self)
 |      businessDayConvention(FxSwapRateHelper self) -> BusinessDayConvention
 |
 |  calendar(self)
 |      calendar(FxSwapRateHelper self) ->

In [54]:
# print(json.dumps(
#     json.loads(curve.model_dump_json()),
#     indent=4,
# ))
crv.SingleCurveConfig.model_validate_json(
    sofr_curve.model_dump_json()
)

2025-09-26 16:56:31,259 - DEBUG - SingleCurveConfig - building quantlib index for SOFR
2025-09-26 16:56:31,260 - DEBUG - SingleCurveConfig - Done building quantlib index


SingleCurveConfig(name='SOFR', ccy=Security(gsid=[USD], ticker=[USD], security_type=[CURRENCY], security_subtype=[NATIONAL_FIAT]), date=datetime.date(2025, 1, 3), settle_days=2, dc=<DayCount.Actual360: 'act/360'>, cal=<Calendar.US_SOFR: 'us/sofr'>, extrapolate=True, virgin=False, bms={'1y': Benchmark(rate=Quote(quote=<QuantLib.QuantLib.SimpleQuote; proxy of <Swig Object of type 'ext::shared_ptr< SimpleQuote > *' at 0x7f38adbe1bf0> >), instrument=Swap(legs=(Leg(ccy=Security(gsid=[USD], ticker=[USD], security_type=[CURRENCY], security_subtype=[NATIONAL_FIAT]), notional=Decimal('10000'), cpn=FixedRate(rate=Decimal('0.0361'), is_constant=True, is_float=False), acc=AccrualInfo(start=datetime.date(2025, 1, 7), end=Period("1Y"), dc=<DayCount.Actual360: 'act/360'>, freq=Decimal('1'), cal_accrual=<Calendar.NULL: 'null'>, cal_pay=<Calendar.US_SOFR: 'us/sofr'>, period=None, bdc=<BusinessDayConvention.following: 'F'>, front_stub_not_back=True, eom=False), pay_delay=None, log=<Logger Leg (DEBUG)>),

In [55]:
curve2 = crv.MultiCurve.model_validate_json(
    curve.model_dump_json()
)

swp_bldr = swp_test_raw.risk_builder(
    # funding_index=curve.get_curve_handle('SOFR'),
    funding_index='SOFR'
)

multi_risk2 = curve2.risk_factory( swp_bldr, eps=1e-5)
print(f'fair rate: {multi_risk2.ql_obj.fairRate():,.3%}')

# multi_risk2 = curve2.risk_factory( swp_bldr, eps=1e-5)
print(f'fair rate: {multi_risk2.ql_obj.fairRate():,.3%}')
print('multi risk pv:', multi_risk2.pv)
df = multi_risk2.curve['SOFR'].as_df()
print(df['risk'].sum())
display(df.style.format({
    'risk':'{0:,.1f}'
}))

2025-09-26 16:56:31,375 - DEBUG - SingleCurveConfig - building quantlib index for SOFR
2025-09-26 16:56:31,376 - DEBUG - SingleCurveConfig - Done building quantlib index
2025-09-26 16:56:31,380 - DEBUG - SingleCurveConfig - building quantlib index for ESTR
2025-09-26 16:56:31,381 - DEBUG - SingleCurveConfig - Done building quantlib index
2025-09-26 16:56:31,382 - DEBUG - MultiCurve - Building all curves
2025-09-26 16:56:31,383 - DEBUG - SingleCurveConfig - Building quantlib curve
2025-09-26 16:56:31,384 - DEBUG - salt.curve - Benchmark building 
2025-09-26 16:56:31,385 - DEBUG - Swap - Building actual ql helper
2025-09-26 16:56:31,387 - DEBUG - salt.curve - Benchmark building 
2025-09-26 16:56:31,388 - DEBUG - Swap - Building actual ql helper
2025-09-26 16:56:31,391 - DEBUG - salt.curve - Benchmark building 
2025-09-26 16:56:31,391 - DEBUG - Swap - Building actual ql helper
2025-09-26 16:56:31,395 - DEBUG - salt.curve - Benchmark building 
2025-09-26 16:56:31,396 - DEBUG - Swap - Build

fair rate: 3.501%
fair rate: 3.501%
multi risk pv: security=Security(gsid=[USD], ticker=[USD], security_type=[CURRENCY], security_subtype=[NATIONAL_FIAT]) quantity=Decimal('-840543.1712559438')
USD    3.511919e+06
dtype: float64


,rate,risk
,,USD
1y,3.610000,352820.252370
2y,3.331000,352544.809451
3y,3.270000,352572.337234
5y,3.311000,352581.755777
7y,3.427000,344613.943467
10y,3.608000,346633.921369
12y,3.717000,352542.015101
20y,3.943000,352563.489498
30y,3.922000,352523.341586


In [56]:
multi_risk2.curve['SOFR'].as_df()


2025-09-26 16:56:33,648 - DEBUG - salt.curve - returning dataframe


,rate,risk
,,USD
1y,3.610,352820.252370
2y,3.331,352544.809451
3y,3.270,352572.337234
5y,3.311,352581.755777
7y,3.427,344613.943467
10y,3.608,346633.921369
12y,3.717,352542.015101
20y,3.943,352563.489498
30y,3.922,352523.341586


In [57]:
help(pd.concat)

Help on function concat in module pandas.core.reshape.concat:

concat(objs: 'Iterable[Series | DataFrame] | Mapping[HashableT, Series | DataFrame]', *, axis: 'Axis' = 0, join: 'str' = 'outer', ignore_index: 'bool' = False, keys: 'Iterable[Hashable] | None' = None, levels=None, names: 'list[HashableT] | None' = None, verify_integrity: 'bool' = False, sort: 'bool' = False, copy: 'bool | None' = None) -> 'DataFrame | Series'
    Concatenate pandas objects along a particular axis.

    Allows optional set logic along the other axes.

    Can also add a layer of hierarchical indexing on the concatenation axis,
    which may be useful if the labels are the same (or overlapping) on
    the passed axis number.

    Parameters
    ----------
    objs : a sequence or mapping of Series or DataFrame objects
        If a mapping is passed, the sorted keys will be used as the `keys`
        argument, unless it is passed, in which case the values will be
        selected (see below). Any None objects

In [58]:
ntnl = 1_000_000
bnd2 = fs.Bond(
    notional=ntnl,
    leg=fs.Leg(
        ccy=usd,
        notional=ntnl,
        cpn=fs.FixedRate(rate=decimal.Decimal('0.00')),
        acc=fs.AccrualInfo(
            start=datetime.date(2025,1,1),
            # end=datetime.date(2026,1,1),
            end='7y',
            dc=fs.DayCount.Thirty360,
            freq=2,
            bdc=fs.BusinessDayConvention.unadjusted,
        )
    ),
    # settle: datetime.date
    settle_days=1
)

bnd_bldr = bnd2.risk_builder( funding_index='SOFR')
multi_risk3 = curve2.risk_factory( bnd_bldr)

# print(f'fair rate: {multi_risk3.ql_obj.fairRate():,.3%}')
# multi_risk2 = curve2.risk_factory( swp_bldr, eps=1e-5)
# print(f'fair rate: {multi_risk3.ql_obj.fairRate():,.3%}')

print('multi risk pv:', multi_risk3.pv)
df2 = multi_risk3.curve['SOFR'].as_df()
print(df2['risk'].sum())
display(df2.style.format({
    'risk':'{0:,.1f}'
}))
print(multi_risk3.ql_obj.cleanPrice())

2025-09-26 16:56:33,709 - DEBUG - MultiCurve - Risk factory
2025-09-26 16:56:33,710 - DEBUG - Bond - funding curve df: 0.223648
2025-09-26 16:56:33,711 - DEBUG - SingleCurveConfig - Building full risk ladder
2025-09-26 16:56:33,712 - DEBUG - SingleCurveConfig - PVing
2025-09-26 16:56:33,713 - DEBUG - SingleCurveConfig - Risking for 1y
2025-09-26 16:56:33,879 - DEBUG - SingleCurveConfig - Risking for 2y
2025-09-26 16:56:34,030 - DEBUG - SingleCurveConfig - Risking for 3y
2025-09-26 16:56:34,193 - DEBUG - SingleCurveConfig - Risking for 5y
2025-09-26 16:56:34,357 - DEBUG - SingleCurveConfig - Risking for 7y
2025-09-26 16:56:34,513 - DEBUG - SingleCurveConfig - Risking for 10y
2025-09-26 16:56:34,652 - DEBUG - SingleCurveConfig - Risking for 12y
2025-09-26 16:56:34,790 - DEBUG - SingleCurveConfig - Risking for 20y
2025-09-26 16:56:34,926 - DEBUG - SingleCurveConfig - Risking for 30y
2025-09-26 16:56:35,060 - DEBUG - SingleCurveConfig - Risking for 40y
2025-09-26 16:56:35,312 - DEBUG - sal

multi risk pv: security=Security(gsid=[USD], ticker=[USD], security_type=[CURRENCY], security_subtype=[NATIONAL_FIAT]) quantity=Decimal('786938.12576816')
USD   -541.12781
dtype: float64


,rate,risk
,,USD
1y,3.610000,1.407440
2y,3.331000,5.521960
3y,3.270000,13.337690
5y,3.311000,23.062350
7y,3.427000,-585.769560
10y,3.608000,-0.283850
12y,3.717000,1.592770
20y,3.943000,0.001130
30y,3.922000,0.001130


78.70157008229106


In [59]:
import cProfile
# cProfile.run('curve2.risk_factory( bnd_bldr, eps=1e-5)', sort='cumtime')

In [60]:
# def new_ccy(name:str, as_reference:bool=True):
#     ccy = fs.FiatCurrency(ticker=name, gsid=name)
#     if as_reference:
#         return fs.create_reference_from_security(ccy)
#     else:
#         return ccy

# usd = new_ccy('USD')
# eur = new_ccy('EUR')
eurusd = fs.Quote(quote=1.18)

fx = salt.fx.FxBoard(
    quotes = {
        (eur.gsid, usd.gsid): eurusd,
    }
)
assert fx.get_fx_quote(eur, usd).value() == 1.18
fx.get_fx_quote(usd, eur).value(),1/1.18

(0.8474576271186441, 0.8474576271186441)

In [61]:
# for x in dir(ql):
#     if 'flat' in x.lower():
#         print(x)
list(fs.DayCount)

[<DayCount.Actual360: 'act/360'>,
 <DayCount.Actual365Fixed: 'act/365f'>,
 <DayCount.ActualActual: 'act/act'>,
 <DayCount.Thirty360: '30/360'>]

In [66]:
fx_swap = fs.Swap(
    legs=(
        fs.Leg(
            ccy=usd,
            notional=1_000_000,
            cpn=fs.FixedRate(rate=decimal.Decimal('1.00')),
            acc=fs.AccrualInfo(
                start=datetime.date(2025,1,1),
                # end=datetime.date(2026,1,1),
                end='2y',
                dc=fs.DayCount.Thirty360,
                freq=0,
                bdc=fs.BusinessDayConvention.unadjusted,
            )
        ),
        fs.Leg(
            ccy=eur,
            notional=-1_000_000,
            cpn=fs.FixedRate(rate=decimal.Decimal('1.00')),
            acc=fs.AccrualInfo(
                start=datetime.date(2025,1,1),
                # end=datetime.date(2026,1,1),
                end='3y',
                dc=fs.DayCount.Thirty360,
                freq=0,
                bdc=fs.BusinessDayConvention.unadjusted,
            )
        ),
    )
)

fl1_bldr = fx_swap.risk_builder(
    # funding_index='SOFR'
)
multi_risk4 = curve2.risk_factory( fl1_bldr )

print('multi risk pv:', multi_risk4.pv)
for k in multi_risk4.curve.keys():
    print(k)
    as_df = multi_risk4.curve[k].as_df()
    print(as_df.sum())
    display(as_df.style.format({
        ('risk','USD'):'{0:,.3f}',
        ('risk','EUR'):'{0:,.3f}',
    }))

2025-09-26 17:00:21,822 - DEBUG - MultiCurve - Risk factory
2025-09-26 17:00:21,824 - DEBUG - Leg - funding curve df: 0.223648
2025-09-26 17:00:21,826 - DEBUG - Leg - funding curve df: 0.416285
2025-09-26 17:00:21,829 - DEBUG - SingleCurveConfig - Building full risk ladder
2025-09-26 17:00:21,830 - DEBUG - SingleCurveConfig - PVing
2025-09-26 17:00:21,831 - DEBUG - SingleCurveConfig - Risking for 1y
2025-09-26 17:00:22,007 - DEBUG - SingleCurveConfig - Risking for 2y


got default funding for ccy USD SOFR
got default funding for ccy EUR ESTR


2025-09-26 17:00:22,160 - DEBUG - SingleCurveConfig - Risking for 3y
2025-09-26 17:00:22,322 - DEBUG - SingleCurveConfig - Risking for 5y
2025-09-26 17:00:22,488 - DEBUG - SingleCurveConfig - Risking for 7y
2025-09-26 17:00:22,643 - DEBUG - SingleCurveConfig - Risking for 10y
2025-09-26 17:00:22,782 - DEBUG - SingleCurveConfig - Risking for 12y
2025-09-26 17:00:22,924 - DEBUG - SingleCurveConfig - Risking for 20y
2025-09-26 17:00:23,061 - DEBUG - SingleCurveConfig - Risking for 30y
2025-09-26 17:00:23,194 - DEBUG - SingleCurveConfig - Risking for 40y
2025-09-26 17:00:23,445 - DEBUG - SingleCurveConfig - Building full risk ladder
2025-09-26 17:00:23,446 - DEBUG - SingleCurveConfig - PVing
2025-09-26 17:00:23,447 - DEBUG - SingleCurveConfig - Risking for 1Y
2025-09-26 17:00:23,545 - DEBUG - SingleCurveConfig - Risking for 2Y
2025-09-26 17:00:23,637 - DEBUG - SingleCurveConfig - Risking for 5Y
2025-09-26 17:00:23,721 - DEBUG - SingleCurveConfig - Risking for 10Y
2025-09-26 17:00:23,812 - 

multi risk pv: positions=[Position(-935887.647 x USD), Position(923507.149 x EUR)]
SOFR
rate          35.95600
risk  USD    182.83002
      EUR      0.00000
dtype: float64


2025-09-26 17:00:23,982 - DEBUG - salt.curve - returning dataframe


ESTR
rate          13.50000
risk  USD      0.00000
      EUR   -273.30221
dtype: float64


In [67]:
print(type(usd), usd)
fix_leg1 = fs.Leg(
    ccy=usd,
    notional=1_000_000,
    cpn=fs.FixedRate(rate=decimal.Decimal('1.00')),
    acc=fs.AccrualInfo(
        start=datetime.date(2025,1,1),
        # end=datetime.date(2026,1,1),
        end='2y',
        dc=fs.DayCount.Thirty360,
        freq=0,
        bdc=fs.BusinessDayConvention.unadjusted,
    )
)

qll1 = fix_leg1.as_quantlib()

fl1_bldr = fix_leg1.risk_builder( funding_index='SOFR')
multi_risk3 = curve2.risk_factory( fl1_bldr )

print('multi risk pv:', multi_risk3.pv)
df2 = multi_risk3.curve['SOFR'].as_df()
print(df2['risk'].sum())
display(df2.style.format({ ('risk','USD'):'{0:,.1f}' }))

# (np.abs(df2["risk"]) > 0).any()
# print(multi_risk3.ql_obj.cleanPrice())

2025-09-26 17:00:43,186 - DEBUG - MultiCurve - Risk factory
2025-09-26 17:00:43,187 - DEBUG - Leg - funding curve df: 0.223648
2025-09-26 17:00:43,188 - DEBUG - SingleCurveConfig - Building full risk ladder
2025-09-26 17:00:43,189 - DEBUG - SingleCurveConfig - PVing
2025-09-26 17:00:43,190 - DEBUG - SingleCurveConfig - Risking for 1y
2025-09-26 17:00:43,364 - DEBUG - SingleCurveConfig - Risking for 2y


<class 'finsec.base.Security'> gsid='USD' ticker='USD' security_type=<SecurityType.CURRENCY: 2> security_subtype=<SecuritySubtype.NATIONAL_FIAT: 505> identifiers=[] primary_exchange=None denominated_ccy=None issuer=None description=None website=None as_of_date=None version_id=None


2025-09-26 17:00:43,516 - DEBUG - SingleCurveConfig - Risking for 3y
2025-09-26 17:00:43,679 - DEBUG - SingleCurveConfig - Risking for 5y
2025-09-26 17:00:43,843 - DEBUG - SingleCurveConfig - Risking for 7y
2025-09-26 17:00:44,003 - DEBUG - SingleCurveConfig - Risking for 10y
2025-09-26 17:00:44,142 - DEBUG - SingleCurveConfig - Risking for 12y
2025-09-26 17:00:44,280 - DEBUG - SingleCurveConfig - Risking for 20y
2025-09-26 17:00:44,416 - DEBUG - SingleCurveConfig - Risking for 30y
2025-09-26 17:00:44,549 - DEBUG - SingleCurveConfig - Risking for 40y
2025-09-26 17:00:44,802 - DEBUG - salt.curve - returning dataframe


multi risk pv: security=Security(gsid=[USD], ticker=[USD], security_type=[CURRENCY], security_subtype=[NATIONAL_FIAT]) quantity=Decimal('-935887.6472021386')
USD    182.8298
dtype: float64


,rate,risk
,,USD
1y,3.610000,-0.0
2y,3.331000,184.3
3y,3.270000,0.0
5y,3.311000,-1.5
7y,3.427000,-0.0
10y,3.608000,-0.0
12y,3.717000,-0.0
20y,3.943000,-0.0
30y,3.922000,-0.0
